# EG 24/25 - Conversão do TPC1 para Lark

## Maria Cunha e Tomás Campinho - Gr10

Link Google Colab: https://colab.research.google.com/drive/1mEA8PuKzugD_vmJPzaZfyOX06TzLaCsB?usp=sharing

## Implementar e adaptar a seguinte gramática em Lark:

Terminal: { '.', ';', '[', ']', num }  
Non-Terminal: { S, Is, RI, I }  
Production Rules (P):  

- p1: Sentence: Signal Intervals '.'  

- p2: Signal: '+'  

- p3: Signal: '-'  

- p4: Intervals: Interval  

- p5: Intervals: Intervals Interval  

- p6: Interval: '[' num ':' num ']'  

  - CC1: p[4] > p[2] &  
  - CC2: p[2] >= parser.anterior  
  - parser.anterior = p[4]  
  - parser.erro = not (CC1) or not (CC2)  


In [1]:
!pip install lark

In [2]:
from lark import Lark, Transformer

grammar = '''
// Regras Sintáticas
sentence: signal interval+ DOT
signal: PLUS
      | MINUS
interval: PE num COL num PD

// Regras Lexicográficas
num: /\d+(\.\d+)?/
PLUS: "+"
MINUS: "-"
DOT: "."
PE:"["
PD:"]"
COL:":"

// Ignorar espaços em branco
%import common.WS
%ignore WS
'''

parser = Lark(grammar, start='sentence', parser='lalr')

class IntervalTransformer(Transformer):
    def __init__(self):
        self.anterior = None
        self.erro = False

    def interval(self, items):
        start = float(items[1].children[0].value)
        end = float(items[3].children[0].value)

        # check CC1 rule: end bound > start bound
        CC1 = end > start
        # check CC2 rule: start bound >= previous end bound
        CC2 = start >= self.anterior if self.anterior is not None else True

        # error handling
        if not CC1:
            print(f"CC1 Error: interval [{start}:{end}] - end bound ({end}) must be greater than start bound ({start}).")
            self.erro = True

        if not CC2:
            print(f"CC2 Error: interval [{start}:{end}] - start bound ({start}) must be greater than or equal to the previous end bound ({self.anterior}).")
            self.erro = True

        # update end bound
        self.anterior = end

    def final_check(self):
        if self.erro:
            print("\nIncorrect expression!")
        else:
            print("The expression is correct!")


input_data = "+   [ 20.0:9.0 ]  [1.5 :19.0] ."
tree = parser.parse(input_data)
print(tree.pretty())

transformer = IntervalTransformer()
res = transformer.transform(tree)
transformer.final_check()

sentence
  signal	+
  interval
    [
    num	20.0
    :
    num	9.0
    ]
  interval
    [
    num	1.5
    :
    num	19.0
    ]
  .

CC1 Error: interval [20.0:9.0] - end bound (9.0) must be greater than start bound (20.0).
CC2 Error: interval [1.5:19.0] - start bound (1.5) must be greater than or equal to the previous end bound (9.0).

Incorrect expression!
